In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(torch.cuda.is_available())

True


In [3]:
class LeNet(nn.Module):

    def __init__(self):

        super(LeNet, self).__init__()

        self.activation = nn.Tanh()
        self.pool = nn.AvgPool2d(kernel_size = (2, 2), stride = (2, 2))
        
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 6, kernel_size = (5, 5), stride = (1, 1), padding = (0, 0))
        self.conv2 = nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = (5, 5), stride = (1, 1), padding = (0, 0))
        self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 120, kernel_size = (5, 5), stride = (1, 1), padding = (0, 0))

        self.fc1 = nn.Linear(in_features = 120, out_features = 84)
        self.fc2 = nn.Linear(in_features = 84, out_features = 10)

    def forward(self, x):

        x = self.activation(self.conv1(x))
        x = self.pool(x)

        x = self.activation(self.conv2(x))
        x = self.pool(x)
        
        x = self.activation(self.conv3(x))

        x = x.reshape(x.shape[0], -1)
        
        x = self.activation(self.fc1(x))
        
        x = self.fc2(x)

        return x

In [4]:
# Hiperparámetros
num_entradas = 1
num_clases = 10
batch_size = 64
learning_rate = 1e-3
num_epocas = 3

In [5]:
# Vamos a descargar el dataset de MNIST
train_dataset = datasets.MNIST(root = 'dataset/', train = True, transform = transforms.ToTensor(), download = True)
train_dataset.data = F.pad(input = train_dataset.data, pad = (2, 2, 2, 2), mode='constant', value=0)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

test_dataset = datasets.MNIST(root = 'dataset/', train = False, transform = transforms.ToTensor(), download = True)
test_dataset.data = F.pad(input = test_dataset.data, pad = (2, 2, 2, 2), mode='constant', value=0)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

In [6]:
print(train_dataset)
print(test_dataset)
print(train_dataset.data.size())
print(test_dataset.data.size())


Dataset MNIST
    Number of datapoints: 60000
    Root location: dataset/
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset MNIST
    Number of datapoints: 10000
    Root location: dataset/
    Split: Test
    StandardTransform
Transform: ToTensor()
torch.Size([60000, 32, 32])
torch.Size([10000, 32, 32])


In [7]:
# Inicializamos el modelo
# Usamos .to(device) para llevar el entrenamiento del modelo al dispositivo disponible (GPU o CPU)
modelo = LeNet().to(device)

In [8]:
# Loss function, para clasificaciones de una sola etiqueta (y) como en este caso podemos utilizar Cross Entropy Loss
func_perdida = nn.CrossEntropyLoss()
optimizador = optim.Adam(params = modelo.parameters(), lr = learning_rate)

In [9]:
cont = 0

for epoca in range(num_epocas):

    correcto = 0
    
    for batch_idx, (x, y) in enumerate(train_loader):

        x = x.to(device = device)
        y = y.to(device = device)

        if (cont == 0):
            
            print(x.shape)
            cont += 1

        # forward
        scores = modelo(x)
        loss = func_perdida(scores, y)

        # Backpropagation
        # Zero your gradients for every batch!
        # Ponemos el gradiente a 0 para cada batch para que no almacene las derivadas de las anteriores capas
        optimizador.zero_grad() 
        loss.backward()

        # Descenso del gradiente o pasos del algoritmo ADAM
        optimizador.step()
        
        #correcto += (scores == y).float().sum()
        _,pred_label = torch.max(scores, dim = 1)
        correcto += (pred_label == y).float().sum()
    
    print(f"Epoca [{epoca+1}/{num_epocas}]\tPrecision {((100 * correcto) / len(train_dataset))}")

torch.Size([64, 1, 32, 32])
Epoca [1/3]	Precision 91.18167114257812
Epoca [2/3]	Precision 97.04666900634766
Epoca [3/3]	Precision 98.0999984741211


In [12]:
# Comprobar precisión del modelo en entrenamiento y validacion
def comprobar_precision(loader, model):

    num_correct = 0
    num_samples = 0
    model.eval()

    # Para evaluar el modelo no necesitamos calcular el gradiente
    with torch.no_grad(): 
    
        for x, y in loader:
    
            x = x.to(device = device)
            y = y.to(device = device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f"Tenemos {num_correct} / {num_samples} con una precision de {float(num_correct) / float(num_samples) * 100:.2f}")

    model.train()

In [11]:
comprobar_precision(train_loader, modelo)
comprobar_precision(test_loader, modelo)

Tenemos 59054 / 60000 con una precision de 98.42
Tenemos 9795 / 10000 con una precision de 97.95
